<a href="https://colab.research.google.com/github/Lewislou/Pytorch-pratical-Learning/blob/master/4_4_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 4: Using GPU acceleration with PyTorch

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 2.8MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [0]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 5.2861e-04,  5.8179e-01,  4.6333e-01,  ...,  2.2627e-01,
         -8.2378e-02,  1.0781e+00],
        [ 2.1575e+00,  1.4869e+00, -1.8420e+00,  ...,  5.1600e-01,
         -3.4442e+00, -5.2721e-03],
        [-6.9793e-02,  3.0324e-01,  3.3978e-01,  ..., -1.3991e+00,
         -3.4825e-01, -2.6279e-02],
        ...,
        [-1.7956e-02,  1.6970e-01, -1.4299e-01,  ...,  6.9882e-02,
         -2.3071e+00,  4.2407e-02],
        [ 6.6980e-02,  7.2593e-01,  1.1960e+00,  ..., -2.0026e-01,
          2.1085e-01, -6.9605e-01],
        [ 1.4919e+00,  2.9736e-01, -1.2249e+00,  ..., -6.7939e-01,
         -1.7923e-01,  7.8726e-01]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [0]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-3.4597e-01, -1.6933e-02,  7.9189e-01,  ...,  5.6216e-03,
          1.0663e-01,  1.7085e-01],
        [-7.4660e-01,  5.7166e-01, -4.9206e-01,  ..., -1.4364e+00,
         -2.4030e-03, -5.1432e-01],
        [ 1.7042e-01,  2.0903e-02, -2.9556e-01,  ...,  3.1874e+00,
          2.4187e-01, -5.2116e-01],
        ...,
        [ 6.8482e-02, -1.3609e-02,  1.6439e+00,  ...,  1.1157e+00,
          7.5659e-03,  4.9428e-01],
        [ 5.8035e-01, -2.4535e-01,  3.2474e-01,  ...,  9.1662e-01,
         -1.4529e-01,  7.3007e-02],
        [-1.1170e+00, -3.7940e+00,  8.8640e-01,  ...,  2.7168e+00,
          9.3212e-01,  2.0537e+00]], device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [0]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-0.2677, -0.0933, -2.1876,  ...,  0.1598, -0.1380,  0.5186],
        [-0.6226,  1.5875, -0.8464,  ...,  0.4358,  1.4805, -0.1445],
        [-0.5830,  0.3000, -0.0639,  ..., -0.4733, -0.2656,  0.5209],
        ...,
        [-0.7534,  2.0859, -0.3019,  ..., -0.0328, -0.1003, -0.1952],
        [ 0.6380, -1.7874, -1.2514,  ...,  0.8244, -0.6818,  0.0619],
        [-1.0866,  1.8005,  0.2937,  ..., -0.6485,  3.2878, -1.0093]],
       device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

YOUR ANSWER HERE\\\
parallel computations.\\\
Nvidia created a parallel computing architecture and platform for its GPUs called CUDA, which gave developers access and the ability to express simple processing operations in parallel through code. 

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

YOUR ANSWER HERE\\\
GPU memory